In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!cd gdrive/MyDrive/ELEC4010N/HW2

In [3]:
# import packages
from __future__ import print_function, division
import math
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchvision import datasets, transforms, utils
from tqdm import tqdm
import torch.optim as optim 
import torch.nn.functional as F
from matplotlib import pyplot as plt
from skimage import io, transform
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [6]:
## Load data
TRANSFORM = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=0.1307,
        std=0.3081
    )
])

TRAIN_DATA_PATH = '/content/gdrive/MyDrive/ELEC4010N/HW2/Train/'
TEST_DATA_PATH = '/content/gdrive/MyDrive/ELEC4010N/HW2/Test/'
train_data = datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM)
entire_trainset = DataLoader(train_data, batch_size=64, shuffle=True)
test_data = datasets.ImageFolder(root=TEST_DATA_PATH, transform=TRANSFORM)
test_dataset = DataLoader(test_data, batch_size=64, shuffle=True)

# split_train_size = int(0.8*(len(entire_trainset)))  # use 80% as train set
# split_valid_size = len(entire_trainset) - split_train_size  # use 20% as validation set

# print(len(entire_trainset))
# print(split_train_size)
# print(split_valid_size)
# train_set, val_set = torch.utils.data.random_split(
#     entire_trainset, [split_train_size, split_valid_size])

# print(f'train set size: {split_train_size}, validation set size: {split_valid_size}')

In [7]:
dataloader_iter = iter(entire_trainset)
batch_data = next(dataloader_iter)
img, label = batch_data
print(img.shape)
print(label.shape)

torch.Size([64, 3, 224, 224])
torch.Size([64])


In [8]:
# for batch in train_data_loader:
#   inputs, targets = batch
#   for img in inputs:
#     print(type(img))
    # image  = img.cpu().numpy()
    # # transpose image to fit plt input
    # image = image.T
    # # normalise image
    # data_min = np.min(image, axis=(1,2), keepdims=True)
    # data_max = np.max(image, axis=(1,2), keepdims=True)
    # scaled_data = (image - data_min) / (data_max - data_min)
    # # show image
    # plt.imshow(image)
    # plt.show()
# for batch in test_data_loader:
#   inputs, targets = batch
#   for img in inputs:
#     image = img.cpu().numpy()
#     image = image.T
#     plt.imshow(image)
#     plt.show()

In [9]:
## define Resnet

def train(model, train_loader, optimizer, Loss):
    model.train()
    
    correct = 0
    total = 0
    train_loss = 0
    for i, data in tqdm(enumerate(train_loader)):
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = model(inputs)
        
        optimizer.zero_grad()
        
        loss = Loss(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, prediction = torch.max(outputs.data, 1)  
        total += labels.size(0)
        correct += (prediction == labels).sum().item()
    train_loss = train_loss / len(train_loader)
    train_acc = 100 * correct / total
    return model, train_loss, train_acc
    
def conv3x3(in_planes, out_planes, stride=1):
  return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                    padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out
    
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, use_fc=False, dropout=None):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.use_fc = use_fc
        self.use_dropout = True if dropout else False
        if self.use_fc:
            print('Using fc.')
            self.fc_add = nn.Linear(512*block.expansion, 512)
        if self.use_dropout:
            print('Using dropout.')
            self.dropout = nn.Dropout(p=dropout)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))
        return nn.Sequential(*layers)

    def forward(self, x, *args):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.mean(dim=(-2, -1))
        x = x.view(x.size(0), -1)
        if self.use_fc:
            x = F.relu(self.fc_add(x))
        if self.use_dropout:
            x = self.dropout(x)
        return x
Resnet50 = ResNet(Bottleneck, [3, 4, 6, 3], dropout=None)

In [10]:
## load ResNet
from torchvision.models import resnet50

pre_trained = resnet50(weights="IMAGENET1K_V2").state_dict()
new_weights = {k: pre_trained[k] for k in Resnet50.state_dict()}
Resnet50.load_state_dict(new_weights)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

<All keys matched successfully>

In [11]:
class Model(nn.Module):
  def __init__(self, encoder, num_classes=10):
    super(Model, self).__init__()
    self.encoder = encoder
    self.classifier = nn.Linear(2048, num_classes)
  def forward(self, x):
    x = self.encoder(x)
    x = self.classifier(x)
    return x
model = Model(Resnet50, 10)

In [12]:
total_epoch = 1
train_losses = []
val_losses = []
train_accs = []
val_accs = []
time_total = 0

model = model.cuda()
lr = 0.1
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
Loss = nn.CrossEntropyLoss()
        
for epoch in range(total_epoch): 
    time_start = time.time()
    model, train_loss, train_acc = train(model, entire_trainset, optimizer, Loss)
    time_end = time.time()
        
    train_losses.append(train_loss)
    train_accs.append(train_acc)
        
    time_duration = round(time_end - time_start, 2)
    time_total += time_duration
        
    print(f'Epoch {epoch+1}, Accuracy(train):{round(train_acc, 2)}, '
              f'Loss(train):{round(train_loss, 4)}, Time: {time_duration}s')

15it [01:14,  4.99s/it]

Epoch 1, Accuracy(train):93.0, Loss(train):0.1634, Time: 74.89s
